In [ ]:
# !pip install datasets
!pip install transformers

In [6]:
import copy
import logging
from dataclasses import dataclass, field
from typing import Dict, Optional, Sequence

import torch
import transformers
# import utils
from torch.utils.data import Dataset
from transformers import Trainer

############################
from datetime import datetime
import os
# import deepspeed
import json, socket
import argparse
from datasets import load_dataset
import datasets
from functools import partial
from typing import Dict, Optional, Sequence, Any
import time

In [7]:
fileslist = {'train': '1_AM_wiki_00'}
# train_data = load_dataset('json', data_files=fileslist, split='train')
train_data = load_dataset('json', data_files=fileslist, split='train', streaming=True)

In [8]:

IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "<s>"
DEFAULT_UNK_TOKEN = "<unk>"

def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            padding="longest",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

def preprocess(
        sources: Sequence[str],
        targets: Sequence[str],
        tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

def tokenize_wiki_batch(
        tokenizer: transformers.PreTrainedTokenizer,
        list_data_dict: Dict[str, Any],
):

    txtinput = "\n{title}\n\n\t\t{text}\n\n"

    # example = list_data_dict
    # print('---DDD---',list_data_dict['text'])
    
    titlelist = list_data_dict['title']
    textlist = list_data_dict['text']
    batchlen = len(list_data_dict['text'])

    sources = [
        f"{tokenizer.bos_token}" for _ in titlelist
    ]
    # targets = [txtinput.format_map(example) if example.get("title", "") != "" else example.get("text", "")
    #           for example in list_data_dict
    #           ]
    
    
    targets = [txtinput.format_map({'title':titlelist[i],'text':textlist[i]}) if textlist[i] != "" else "\n"
              for i in range(batchlen)
              ]
        

    # logging.warning("Tokenizing inputs... This may take some time...")
    print('---DDD---',len(sources),len(targets))
    data_dict = preprocess(sources, targets, tokenizer)
    input_ids = data_dict["input_ids"]
    labels = data_dict["labels"]

    return {
        "input_ids": input_ids,
        "labels": labels,
    }


In [9]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    'TheBloke/Llama-2-7B-fp16',
    cache_dir='',
    model_max_length=2048,
    padding_side="right",
    use_fast=False,
    legacy=False ############ p5 test
)

tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# buffer_size not allow for batch mode
train_dataset = train_data.shuffle(10000).map(
    partial(
        # tokenize_prompt_gen,
        tokenize_wiki_batch,
        tokenizer
    ),
    remove_columns=['id','url','title','text'],
    batched=True
)

# @dataclass
# class DataCollatorForSupervisedDataset(object):
#     """Collate examples for supervised fine-tuning."""

#     tokenizer: transformers.PreTrainedTokenizer

#     def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
#         input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))

#         input_ids = torch.Tensor([torch.nn.utils.rnn.pad_sequence(
#             input_id, batch_first=True, padding_value=self.tokenizer.pad_token_id) for input_id in input_ids])
#         labels = [torch.nn.utils.rnn.pad_sequence(label, batch_first=True, padding_value=IGNORE_INDEX) for label in labels]
#         attention_mask = [input_id.ne(self.tokenizer.pad_token_id) for input_id in input_ids]

#         # input_ids = torch.nn.utils.rnn.pad_sequence(
#         #     input_ids[0], batch_first=True, padding_value=self.tokenizer.pad_token_id)
#         # labels = torch.nn.utils.rnn.pad_sequence(labels[0], batch_first=True, padding_value=IGNORE_INDEX)
#         # attention_mask = input_ids.ne(self.tokenizer.pad_token_id)

#         return dict(
#             input_ids=input_ids,
#             labels=labels,
#             attention_mask=attention_mask,
#         )

    
# data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
# # data_module = dict(train_dataset=train_data, eval_dataset=None, data_collator=data_collator)
# data_module = dict(train_dataset=train_data.with_format("torch"), eval_dataset=None, data_collator=data_collator)



In [10]:
next(iter(train_dataset))


---DDD--- 790 790


{'input_ids': tensor([    1,     1,    13, 30313, 30319,   313, 31046, 30732, 29897,    13,
            13,    12,    12, 30313, 30319,   313, 31046, 30732, 29897,    13,
            13, 30866, 30313, 30319, 30843, 30419, 30409, 30214,   232,   146,
           139, 31685, 30866, 31203, 30313, 30577, 30319, 30843, 31391, 30866,
           233,   139,   183,   236,   190,   148,   232,   187,   192, 30210,
         31203, 30313, 30843, 30214, 30392, 29896, 29929, 29953, 29946, 30470,
         31419, 31107, 30594, 31480, 31424, 31195, 30888, 31349, 31046, 30613,
           236,   158,   186, 30728, 30064, 31530, 31168, 31107, 31141,   234,
           190,   155, 31072, 30210, 30287,   232,   188,   136, 31046, 30732,
         30267,   232,   177,   134, 30392, 30287,   232,   188,   136, 30688,
         31046, 31551, 30214,   234,   190,   155, 31072, 30210, 30392, 30287,
         30502,   233,   139,   183,   235,   148,   154,   232,   159,   137,
           236,   164,   185,   234,   

In [11]:
import numpy
numpy.__version__

'1.24.4'